In [36]:
import pandas as pd
import numpy as np

22.12기준 인구집계현황 데이터 - 인구소멸 위기지역 Top10 구하기

- 기초자치단체(시, 군)
특례시: 수원, 고양, 용인(구)
행정구: 성남, 안산, 안양(구)

In [37]:
df = pd.read_csv('data/주민등록인구집계현황.csv',encoding='euc=kr')
df.head(1)

,연도,월,행정구역구분명,행정구역명,총 인구수,0~9세,10~19세,20~29세,30~39세,40~49세,...,10~19세 (여),20~29세 (여),30~39세 (여),40~49세 (여),50~59세 (여),60~69세 (여),70~79세 (여),80~89세 (여),90~99세 (여),100세 이상 (여)
0,2022,12,도,경기도,13589432,1040954,1327726,1737291,1882300,2284603,...,647283,821150,904073,1121889,1133428,878494,437251,252711,44184,1369


In [38]:
# 카테고리 정리/ 셀렉
df = df[['행정구역구분명','행정구역명','20~29세 (여)','30~39세 (여)','60~69세','70~79세','80~89세','90~99세','100세 이상']]
df.head(2)

,행정구역구분명,행정구역명,20~29세 (여),30~39세 (여),60~69세,70~79세,80~89세,90~99세,100세 이상
0,도,경기도,821150,904073,1750962,808663,403721,58065,1725
1,시군,경기도 가평군,2503,2433,13315,7082,3999,564,19


In [39]:
# 인구소멸비율 컬럼 만들고 값 구하기
df['가임기여성수'] = df['20~29세 (여)'] + df['30~39세 (여)']
df['노령인구'] = df.loc[:, '60~69세':'100세 이상'].sum(axis=1)
df['인구소멸비율'] = df['가임기여성수'] / df['노령인구']        # iloc[:, 2:7].sum(axis=1) / iloc[:, -2:].sum(axis=1).round(2)
df.head(2)

,행정구역구분명,행정구역명,20~29세 (여),30~39세 (여),60~69세,70~79세,80~89세,90~99세,100세 이상,가임기여성수,노령인구,인구소멸비율
0,도,경기도,821150,904073,1750962,808663,403721,58065,1725,1725223,3023136,0.570673
1,시군,경기도 가평군,2503,2433,13315,7082,3999,564,19,4936,24979,0.197606


In [40]:
# 4) 원하는 행만 (Filtering)
df=df[['행정구역구분명','행정구역명','인구소멸비율']]
df.head(2)

,행정구역구분명,행정구역명,인구소멸비율
0,도,경기도,0.570673
1,시군,경기도 가평군,0.197606


In [53]:
# df= df[df.행정구역구분명.str.contains('시군') | df.행정구역구분명.str.contains('구')]
# df.head(30)

,행정구역구분명,행정구역명,인구소멸비율
1,시군,경기도 가평군,0.197606
8,시군,경기도 고양시,0.570036
9,구,경기도 고양시 덕양구,0.555132


In [54]:
# 화영언니 코드- isin() : 똑같은 값만 True / 부분일치나 포함은 False

# 시군, 구 데이터 이면서 &
# 특정 값(도, 특례시, 광역시..)은 아닌(~) 데이터
except_city = ['경기도 수원시', '경기도 고양시', '경기도 용인시', '경기도 성남시', '경기도 안산시', '경기도 안양시']
df[((df.행정구역구분명 == '시군') | (df.행정구역구분명 == '구')) & ~(df['행정구역명'].str.strip().isin(except_city))]
df.head(5)

,행정구역구분명,행정구역명,인구소멸비율
1,시군,경기도 가평군,0.197606
8,시군,경기도 고양시,0.570036
9,구,경기도 고양시 덕양구,0.555132
31,구,경기도 고양시 일산동구,0.604745
44,구,경기도 고양시 일산서구,0.561588


In [43]:
# 인구소멸위기 지역 Top 10
df.sort_values(by='인구소멸비율').head(10)

,행정구역구분명,행정구역명,인구소멸비율
1,시군,경기도 가평군,0.197606
391,시군,경기도 양평군,0.199129
417,시군,경기도 연천군,0.203053
404,시군,경기도 여주시,0.268597
558,시군,경기도 포천시,0.276898
156,시군,경기도 동두천시,0.325018
328,시군,경기도 안성시,0.381931
363,구,경기도 안양시 만안구,0.482571
464,구,경기도 용인시 처인구,0.484903
218,구,경기도 성남시 중원구,0.496542


In [44]:
# 안전 지역 Top 10
df.sort_values(by='인구소멸비율', ascending=False).head(10)

,행정구역구분명,행정구역명,인구소멸비율
589,시군,경기도 화성시,0.868272
428,시군,경기도 오산시,0.763604
231,시군,경기도 수원시,0.739027
280,시군,경기도 시흥시,0.722085
574,시군,경기도 하남시,0.667054
177,구,경기도 성남시 분당구,0.664424
452,구,경기도 용인시 수지구,0.663773
532,시군,경기도 평택시,0.654703
301,구,경기도 안산시 단원구,0.642797
300,시군,경기도 안산시,0.632807
